In [1]:
%run ./WaveSetEnvironment1

In [2]:
import pyspark.sql.functions as f

In [3]:
# SLA Data
sla_data = spark.read.format('csv') \
  .option("header", 'true') \
  .option("sep", ',') \
  .option("inferSchema", "true") \
  .option("dateFormat","yyyy/MM/dd HH")\
  .load(SLA_PATH) \
  .withColumnRenamed("PickTicketCreateDateTime","Date") \
  .withColumnRenamed("At Once","AtOnce") \
  .drop("PickTicketControlNumber").drop("StartShipdateKey")

# #Aggregations of SLA Data
sla_staging_sum = sla_data.groupBy("WaveNumber") \
  .sum("SLAMet")
display(sla_data)
# # sla_staging_agg = sla_data.groupBy("WaveNumber") \

#   .avg("Month")\
#   .avg("Year")\
#   .avg("Weekday")\
#   .avg("Week")\
#   .avg("WKDay")\
#   .avg("Hour")\
#   .avg("Current")\
#   .avg("Future")\
#   .avg("AtOnce")
  

# sla_staging_count = sla_data.groupBy("WaveNumber") 
#   .agg(lit(func.count("SLAMet")).alias("count_alias")) 
#   .agg(func.("Month"))
#   .agg(lit(func.avg("Year")))\
#   .agg(lit(func.avg("Weekday")))\
#   .agg(lit(func.avg("Week")))\
#   .agg(lit(func.avg("WKDay")))\
#   .agg(lit(func.avg("Hour")))\
#   .agg(lit(func.avg("Current")))\
#   .agg(lit(func.avg("Future")))\
#   .agg(lit(func.avg("AtOnce")))
  
# sla_staging = sla_staging_sum.join(sla_staging_count,"WaveNumber") \
#   .withColumn("Ratio",col("sum(SLAMet)")/col("count_alias")) \
#   .withColumn("y", when(col("Ratio") < 1.0, 0).otherwise(1)) \
#   .drop("sum(SLAMet)").drop("count_alias")


# #Joining SLAback
# sla_data = sla_staging.join(sla_data, "WaveNumber")
# display(sla_data)

#wave data
wave_data = spark.read.format('csv') \
  .option("header", 'true') \
  .option("sep", ',') \
  .option("inferSchema", "true") \
  .option("dateFormat","yyyy-MM-dd HH") \
  .load(WAVE_PATH) \
  .drop("WaveType").drop("WaveTypeDesc").drop("WaveStatus").drop("ModificationDateTime").drop("_c0")
# Join SLA to Wave
wave_to_lake =wave_data.join(sla_data, "WaveNumber")
display(wave_data)

              
              


In [4]:
#Throughput
 
ThroughPut = spark.read.format('csv') \
  .option("header", 'true') \
  .option("sep", ',') \
  .option("inferSchema", "true") \
  .option("dateFormat","yyyy/MM/dd HH:mm")\
  .load(ThroughPut_Path) \
  .withColumnRenamed("EndDateTime","Date") \
  .withColumnRenamed("APlusStocking","TPAPlusStocking") \
  .withColumnRenamed("APlusPicking","TPAPlusPicking") \
  .withColumnRenamed("BatchWall","TPBatchWall") \
  .withColumnRenamed("HighBay","TPHighBay") \
  .withColumnRenamed("SpecialHandling","TPSpecialHandling")\
  .withColumn("TotalThroughPut",(col("TPAPlusPicking")+col("TPSpecialHandling")/2)/100)\
  .drop("_c0")
display(ThroughPut)

In [5]:
#Join SLA/WaveData to Throughput
data_to_lake = ThroughPut.join(wave_to_lake, "Date")

display(data_to_lake)
# #Write Main Data Set
data_to_lake.write.format('delta') \
  .mode("overwrite") \
  .option('overwriteSchema', 'true') \
  .save(DIR + 'ThroughPut_Train_Dtt.delta')

In [6]:
data_to_lake